In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/cleaned_review_beauty.csv')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Split the data into training and testing sets for TF-IDF
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(
    df['clean_text'], df['rating_class'], test_size=0.2, random_state=42
)

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the training data
X_train_tfidf_features = tfidf_vectorizer.fit_transform(X_train_tfidf)

# Transform the test data
X_test_tfidf_features = tfidf_vectorizer.transform(X_test_tfidf)

print("TF-IDF Features Shape (Train):", X_train_tfidf_features.shape)
print("TF-IDF Features Shape (Test):", X_test_tfidf_features.shape)

TF-IDF Features Shape (Train): (22860, 5000)
TF-IDF Features Shape (Test): (5716, 5000)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize Logistic Regression model
logistic_regression_model = LogisticRegression(max_iter=1000)

# Train the model with TF-IDF features
logistic_regression_model.fit(X_train_tfidf_features, y_train_tfidf)

# Predict on the test data
y_pred_tfidf = logistic_regression_model.predict(X_test_tfidf_features)

# Calculate accuracy
accuracy_tfidf = accuracy_score(y_test_tfidf, y_pred_tfidf)
print(f"Logistic Regression Accuracy with TF-IDF: {accuracy_tfidf}")

Logistic Regression Accuracy with TF-IDF: 0.8254023792862142


In [ ]:

import numpy as np
import gensim
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load dataset
df = pd.read_csv("cleaned_review_beauty.csv")

# Filter and encode sentiment
df = df[df['rating_class'].isin(['good', 'bad'])]
df['sentiment'] = df['rating_class'].map({'good': 1, 'bad': 0})

# Tokenize the clean text
sentences = [str(text).split() for text in df['clean_text']]
y = df['sentiment'].values

# Train Word2Vec
w2v_model = gensim.models.Word2Vec(sentences, vector_size=100, window=5, min_count=2)
w2v_model.train(sentences, total_examples=len(sentences), epochs=10)

# Use Keras Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['clean_text'])
sequences = tokenizer.texts_to_sequences(df['clean_text'])
word_index = tokenizer.word_index

# Pad sequences
max_len = 100
X = pad_sequences(sequences, maxlen=max_len)

# Create embedding matrix
embedding_dim = 100
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
    else:
        embedding_matrix[i] = np.random.normal(0, 0.1, embedding_dim)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix],
                    input_length=max_len, trainable=False))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train
model.fit(X_train, y_train, batch_size=64, epochs=5, validation_split=0.1)

# Evaluate
y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()
print("Accuracy:", accuracy_score(y_test, y_pred))


ModuleNotFoundError: No module named 'gensim'

In [ ]:
%pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 re

In [ ]:
import numpy as np
import gensim
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Filter and encode sentiment
df = df[df['rating_class'].isin(['good', 'bad'])]
df['sentiment'] = df['rating_class'].map({'good': 1, 'bad': 0})

# Tokenize the clean text
sentences = [str(text).split() for text in df['clean_text']]
y = df['sentiment'].values

# Train Word2Vec
w2v_model = gensim.models.Word2Vec(sentences, vector_size=100, window=5, min_count=2)
w2v_model.train(sentences, total_examples=len(sentences), epochs=10)

# Use Keras Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['clean_text'])
sequences = tokenizer.texts_to_sequences(df['clean_text'])
word_index = tokenizer.word_index

# Pad sequences
max_len = 100
X = pad_sequences(sequences, maxlen=max_len)

# Create embedding matrix
embedding_dim = 100
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
    else:
        embedding_matrix[i] = np.random.normal(0, 0.1, embedding_dim)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix],
                    input_length=max_len, trainable=False))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train
model.fit(X_train, y_train, batch_size=64, epochs=5, validation_split=0.1)

# Evaluate
y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()
print("Accuracy:", accuracy_score(y_test, y_pred))

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
import numpy as np
import gensim
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Filter and encode sentiment
df = df[df['rating_class'].isin(['good', 'bad'])]
df['sentiment'] = df['rating_class'].map({'good': 1, 'bad': 0})

# Tokenize the clean text
sentences = [str(text).split() for text in df['clean_text']]
y = df['sentiment'].values

# Train Word2Vec
w2v_model = gensim.models.Word2Vec(sentences, vector_size=100, window=5, min_count=2)
w2v_model.train(sentences, total_examples=len(sentences), epochs=10)

# Use Keras Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['clean_text'])
sequences = tokenizer.texts_to_sequences(df['clean_text'])
word_index = tokenizer.word_index

# Pad sequences
max_len = 100
X = pad_sequences(sequences, maxlen=max_len)

# Create embedding matrix
embedding_dim = 100
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
    else:
        embedding_matrix[i] = np.random.normal(0, 0.1, embedding_dim)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix],
                    input_length=max_len, trainable=False))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train
model.fit(X_train, y_train, batch_size=64, epochs=5, validation_split=0.1)

# Evaluate
y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()
print("Accuracy:", accuracy_score(y_test, y_pred))

/usr/local/lib/python3.11/dist-packages/gensim/utils.py:35: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.3.1)
  import scipy.sparse


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
%pip install --upgrade --force-reinstall gensim numpy

  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached numpy-2.3.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting 

In [ ]:
import numpy as np
import gensim
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Filter and encode sentiment
df = df[df['rating_class'].isin(['good', 'bad'])]
df['sentiment'] = df['rating_class'].map({'good': 1, 'bad': 0})

# Tokenize the clean text
sentences = [str(text).split() for text in df['clean_text']]
y = df['sentiment'].values

# Train Word2Vec
w2v_model = gensim.models.Word2Vec(sentences, vector_size=100, window=5, min_count=2)
w2v_model.train(sentences, total_examples=len(sentences), epochs=10)

# Use Keras Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['clean_text'])
sequences = tokenizer.texts_to_sequences(df['clean_text'])
word_index = tokenizer.word_index

# Pad sequences
max_len = 100
X = pad_sequences(sequences, maxlen=max_len)

# Create embedding matrix
embedding_dim = 100
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
    else:
        embedding_matrix[i] = np.random.normal(0, 0.1, embedding_dim)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix],
                    input_length=max_len, trainable=False))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train
model.fit(X_train, y_train, batch_size=64, epochs=5, validation_split=0.1)

# Evaluate
y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()
print("Accuracy:", accuracy_score(y_test, y_pred))

/tmp/ipython-input-6-377918342.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['rating_class'].map({'good': 1, 'bad': 0})
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 94s 310ms/step - accuracy: 0.8895 - loss: 0.2965 - val_accuracy: 0.9261 - val_loss: 0.1906
Epoch 2/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 83s 300ms/step - accuracy: 0.9249 - loss: 0.1916 - val_accuracy: 0.9327 - val_loss: 0.1627
Epoch 3/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 141s 296ms/step - accuracy: 0.9386 - loss: 0.1593 - val_accuracy: 0.9347 - val_loss: 0.1498
Epoch 4/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 81s 294ms/step - accuracy: 0.9482 - loss: 0.1366 - val_accuracy: 0.9465 - val_loss: 0.1352
Epoch 5/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 83s 300ms/step - accuracy: 0.9556 - loss: 0.1177 - val_accuracy: 0.9475 - val_loss: 0.1314
154/154 ━━━━━━━━━━━━━━━━━━━━ 10s 62ms/step
Accuracy: 0.9481844145246838
